In [ ]:
import os, sys
# ensure project root is on PYTHONPATH
ROOT = os.path.abspath("..")       # adjust if needed
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

from task_dataset import TaskDataset

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import sys
import os

# Set paths
EMB_PATH = "F:\\TML_model_stealing\\TML25_A2_39\\data\\public_embeddings_seed61437433_port9732.npz"
IMG_PATH = "F:\\TML_model_stealing\\TML25_A2_39\\data\\ModelStealingPub.pt"

print("Loading from:")
print(" - Embeddings:", EMB_PATH)
print(" - Images:", IMG_PATH)


In [ ]:
# Load the victim .npz
data = np.load(EMB_PATH)
indices = data["indices"]
reps = data["reps"]
seed = int(data["seed"])
port = int(data["port"])

print("Loaded embeddings:", reps.shape)

# Load the corresponding public image dataset (.pt)
dataset = torch.load(IMG_PATH, weights_only=False)
images  = [dataset.imgs[i] for i in indices]

# Ensure length matches
assert len(images) == reps.shape[0]
print("Loaded matching images:", len(images))


In [ ]:
# Show first 5 images and their first 20 embedding dims
for i in range(5):
    img = images[i]
    emb = reps[i]

    plt.figure(figsize=(4, 1.5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.plot(emb[:20])
    plt.title("Embedding dims 0–19")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# Define Stolen Encoder Architecture

import torch
import torch.nn as nn
from torchvision.models import resnet18

class StolenEncoder(nn.Module):
    def __init__(self, pretrained_backbone=False):
        super().__init__()
        # 1) Backbone
        self.backbone = resnet18(pretrained=pretrained_backbone)
        # Remove the original classification head
        self.backbone.fc = nn.Identity()
        # 2) Projection head to get 1024-dim outputs
        self.proj = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1024)
        )

    def forward(self, x):
        # x shape: (B,3,32,32)
        feat = self.backbone(x)     # (B,512)
        rep  = self.proj(feat)      # (B,1024)
        return rep

# Quickly test shape
model = StolenEncoder(pretrained_backbone=False)
x = torch.randn(4, 3, 32, 32)
y = model(x)
print("➡️ StolenEncoder output shape:", y.shape)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Basic transforms to convert PIL → Tensor + normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

class StealingDataset(Dataset):
    def __init__(self, images, embeddings, transform=None):
        """
        images: list of PIL.Image
        embeddings: numpy array (N,1024)
        """
        self.images     = images
        self.embeddings = torch.from_numpy(embeddings).float()
        self.transform  = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        if img.mode != 'RGB':
            img = img.convert("RGB")
        if self.transform:
            img = self.transform(img)
        rep = self.embeddings[idx]
        return img, rep


# Instantiate dataset & dataloader
dataset_ssl = StealingDataset(images, reps, transform=transform)
loader_ssl  = DataLoader(dataset_ssl, batch_size=64, shuffle=True, num_workers=2)

print("Dataset size:", len(dataset_ssl))
# print("One batch shapes:", next(iter(loader_ssl))[0].shape, next(iter(loader_ssl))[1].shape)


In [ ]:
# Set up training components

import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = StolenEncoder(pretrained_backbone=False).to(device)

# Loss: MSE between student & victim embeddings
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# (Optional) learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


In [ ]:
# Train for one epoch & log loss

model.train()
running_loss = 0.0

for batch_idx, (imgs, target_reps) in enumerate(loader_ssl, 1):
    imgs = imgs.to(device)                  # (B,3,32,32)
    target_reps = target_reps.to(device)    # (B,1024)

    optimizer.zero_grad()
    pred_reps = model(imgs)                 # (B,1024)
    loss = criterion(pred_reps, target_reps)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 50 == 0:
        print(f"  [Batch {batch_idx}] Avg Loss: {running_loss / batch_idx:.4f}")

# Step the scheduler
scheduler.step()

epoch_loss = running_loss / len(loader_ssl)
print(f"→ Epoch Loss: {epoch_loss:.4f}")


## SimCLR

#### Imports and setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

# Path to victim embeddings and image dataset
EMB_PATH = "F:\\TML_model_stealing\\TML25_A2_39\\data\\public_embeddings_seed61437433_port9732.npz"
IMG_PATH = "F:\\TML_model_stealing\\TML25_A2_39\\data\\ModelStealingPub.pt"


In [ ]:
# Load victim embeddings and image indices
data = np.load(EMB_PATH)
reps = data["reps"]       # shape: (N,1024)
indices = data["indices"]

In [ ]:
# Load full dataset and extract relevant images
full_dataset = torch.load(IMG_PATH, weights_only=False)
images = [full_dataset.imgs[i] for i in indices]

print(f"Loaded {len(images)} images and {reps.shape[0]} victim embeddings")

#### Define Augmentations and Dataset

In [ ]:
# SimCLR augmentations (strong)
simclr_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.2, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# Dataset class returning two views + victim rep
class SimCLRDataset(Dataset):
    def __init__(self, images, victim_reps, transform):
        self.images = images
        self.vreps = torch.from_numpy(victim_reps).float()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        if img.mode != 'RGB':
            img = img.convert("RGB")
        t1 = self.transform(img)
        t2 = self.transform(img)
        rep = self.vreps[idx]
        return t1, t2, rep

#### Visualize Augmentations

In [ ]:
# Show original image and two augmented views
img_example = images[10]
aug1 = simclr_transform(img_example)
aug2 = simclr_transform(img_example)

# Convert tensors back to image format for display
def deprocess(t):
    t = t.clone().detach().permute(1,2,0)
    t = t * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
    return t.clamp(0, 1).numpy()

plt.figure(figsize=(10,3))
plt.subplot(1,3,1)
plt.imshow(img_example)
plt.title("Original")
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(deprocess(aug1))
plt.title("Aug View 1")
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(deprocess(aug2))
plt.title("Aug View 2")
plt.axis('off')
plt.tight_layout()
plt.show()

#### Define Encoder Model

In [ ]:
from torchvision.models import resnet18

class StolenEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(pretrained=False)
        self.backbone.fc = nn.Identity()
        self.head = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1024)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

#### Train SimCLR Loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = StolenEncoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loader = DataLoader(SimCLRDataset(images, reps, simclr_transform), batch_size=64, shuffle=True)

alpha = 0.5  # weight between victim alignment and self-consistency
num_epochs = 10
loss_history = []

for epoch in range(1, num_epochs+1):
    model.train()
    total = 0
    for t1, t2, yv in loader:
        t1, t2, yv = t1.to(device), t2.to(device), yv.to(device)

        optimizer.zero_grad()
        o1 = model(t1)         # (B,1024)
        o2 = model(t2)         # (B,1024)
        loss_A = F.mse_loss(o1, yv)
        loss_B = F.mse_loss(o2, o1.detach())
        loss = alpha * loss_A + (1 - alpha) * loss_B
        loss.backward()
        optimizer.step()
        total += loss.item()

    avg = total / len(loader)
    loss_history.append(avg)
    print(f"[Epoch {epoch}] Loss: {avg:.4f}")

# Plot training loss
plt.plot(loss_history, marker='o')
plt.title("SimCLR-style Stealing Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()


In [ ]:
import torch.onnx

ONNX_PATH = "F:\\TML_model_stealing\\TML25_A2_39\\models\\model_Sim_CLR.onnx"
dummy_input = torch.randn(1, 3, 32, 32).to(device)

torch.onnx.export(
    model,
    dummy_input,
    ONNX_PATH,
    input_names=["x"],
    output_names=["embedding"],
    export_params=True,
    opset_version=11,
    do_constant_folding=True
)

print(f"\u2705 Exported SimCLR ONNX model to: {ONNX_PATH}")